[View in Colaboratory](https://colab.research.google.com/github/idleuncle/Colab/blob/master/ColabTemplate.ipynb)

# Colab Template



## 一、Colab 环境准备

1. 在Colab中打开ColabTemplate.ipynb，另存为你的项目MyProject.ipynb并打开。

2. "Colab 环境准备" 完成以下工作，只需要项目打开 时执行一次。

    - 安装系统依赖

    - 授权登录 Google Drive

    - 安装 Colab 编程环境支持包 (IpynbImporter.py, [ColabModules.ipynb](https://colab.research.google.com/drive/1IMv93f2bMYhrx2lfL3cmDBI7kmjCMy01#scrollTo=VyDM84dOxu18))
    
3. 修改并保存ColabModules.ipynb后，执行“下载 Colab 编程环境支持包”及“导入 Colab 编程环境支持包”。

drive变量指向登录的Google Drive。

### 1.1 授权登录Google Drive

#### 第一次授权登录Google Drive

In [0]:
# 安装 PyDrive 操作库，该操作每个 notebook 只需要执行一次
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def google_drive_login():
  # 授权登录，仅第一次的时候会鉴权
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  return drive

drive = google_drive_login()

#### 第二次授权映射Google Drive至本地driver目录

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials

creds = GoogleCredentials.get_application_default()

In [0]:
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls -lt drive

### 1.2 下载 Colab 编程环境支持包

In [0]:
import os
def google_drive_download_files(drive, file_name_prefix, colab_dir=".", overwrite=True):
  # choose a local (colab) directory to store the data.
  local_download_path = os.path.expanduser(colab_dir)
  try:
    os.makedirs(local_download_path)
  except: pass

  # 2. Auto-iterate using the query syntax
  #    https://developers.google.com/drive/v2/web/search-parameters
  file_list = drive.ListFile(
      {'q': "title contains '%s'" % (file_name_prefix) }).GetList()

  files_dict = {}
  for f in file_list:
    # 3. Create & download by id.
    print('title: %s, id: %s' % (f['title'], f['id']))
    fname = os.path.join(local_download_path, f['title'])
    if overwrite or not os.path.exists(fname):
      print('downloading to {}'.format(fname))
      f_ = drive.CreateFile({'id': f['id']})
      f_.GetContentFile(fname)
      print('Download Completed!')
    files_dict[ f['title'] ] = fname

  # with open(fname, 'r') as f:
  #   print(f.read())
  return files_dict, local_download_path

# 修改完ColabModles.ipynb后，执行以下命令，(并在项目中执行菜单项 Runtime/Restart runtime ???)
google_drive_download_files(drive, 'IpynbImporter.py')
google_drive_download_files(drive, 'ColabModules.ipynb')

!ls -lt


### 1.3 导入Colab基础编程环境支持包

In [0]:
import IpynbImporter
from ColabModules import *

colab_ready()

!hostname
!ls -lt

## 二、开始研究代码